In [122]:
import requests
import json
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import statsmodels.api as sm
import statsmodels.stats.api as sms
import statsmodels.formula.api as smf
import scipy.stats as stats
from geopy.geocoders import Nominatim
import time

In [224]:
geolocator = Nominatim(user_agent="grace")
location = geolocator.reverse("40.84916900104506, -73.86835600032798")
print(location.address)

Bronx Park East, Birchall Avenue, Bronxdale, The Bronx, Bronx County, NYC, New York, 10462, USA


In [169]:
import uszipcode
from uszipcode import Zipcode
from uszipcode import SearchEngine
search = SearchEngine(simple_zipcode=False)

In [234]:
def coord_to_zip(lat, long, radius=3, returns=1):
    result = search.by_coordinates(float(lat), float(long), radius=radius, returns=returns)
    try:
        return result[0].zipcode
    except:
        return "No zipcode found"

In [164]:
def address_from_coord(lat, long):
    start = time.time()
    geolocator = Nominatim(user_agent="jdmitchell")
    location = geolocator.reverse(f'{lat}, {long}')
    if time.time()-start<1:
        time.sleep(1.01-(time.time()-start))
    return location.address

In [165]:
def zipcode_from_coord(lat, long):
    return address_from_coord(lat, long).split(", ")[-2]

In [237]:
address_from_coord(40.84916900104506, -73.86835600032798)

'Bronx Park East, Birchall Avenue, Bronxdale, The Bronx, Bronx County, NYC, New York, 10462, USA'

In [137]:
zipcode_from_coord(40.84916900104506, -73.86835600032798)

'10462'

In [13]:
df_subway = pd.read_csv("subway_entrances.csv")

In [82]:
df_subway['latlong']=df_subway['the_geom'].str[7:-1]

In [83]:
df_subway[df_subway.latlong == '-73.86835600032798 40.84916900104506']

,OBJECTID,URL,NAME,the_geom,LINE,latlong,lat,long
0,1734,http://web.mta.info/nyct/service/,Birchall Ave & Sagamore St at NW corner,POINT (-73.86835600032798 40.84916900104506),2-5,-73.86835600032798 40.84916900104506,40.84916900104506,-73.86835600032798


In [115]:
df_exerpt_subway = df_subway[:10]

In [121]:
df_exerpt_subway

,OBJECTID,URL,NAME,the_geom,LINE,latlong,lat,long,zipcode
0,1734,http://web.mta.info/nyct/service/,Birchall Ave & Sagamore St at NW corner,POINT (-73.86835600032798 40.84916900104506),2-5,"[-73.86835600032798, 40.84916900104506]",40.84916900104506,-73.86835600032798,10462
1,1735,http://web.mta.info/nyct/service/,Birchall Ave & Sagamore St at NE corner,POINT (-73.86821300022677 40.84912800131844),2-5,"[-73.86821300022677, 40.84912800131844]",40.84912800131844,-73.86821300022677,10462
2,1736,http://web.mta.info/nyct/service/,Morris Park Ave & 180th St at NW corner,POINT (-73.87349900050798 40.84122300105249),2-5,"[-73.87349900050798, 40.84122300105249]",40.84122300105249,-73.87349900050798,10460
3,1737,http://web.mta.info/nyct/service/,Morris Park Ave & 180th St at NW corner,POINT (-73.8728919997833 40.84145300067447),2-5,"[-73.8728919997833, 40.84145300067447]",40.84145300067447,-73.8728919997833,10460
4,1738,http://web.mta.info/nyct/service/,Boston Rd & 178th St at SW corner,POINT (-73.87962300013866 40.84081500075867),2-5,"[-73.87962300013866, 40.84081500075867]",40.84081500075867,-73.87962300013866,10460
5,1739,http://web.mta.info/nyct/service/,Boston Rd & E Tremont Ave at NW corner,POINT (-73.88000500027815 40.840434000875874),2-5,"[-73.88000500027815, 40.840434000875874]",40.840434000875874,-73.88000500027815,10460
6,1740,http://web.mta.info/nyct/service/,Boston Rd & E Tremont Ave at NE corner,POINT (-73.87983300021861 40.84035400111976),2-5,"[-73.87983300021861, 40.84035400111976]",40.84035400111976,-73.87983300021861,10460
7,1741,http://web.mta.info/nyct/service/,Boston Rd & 178th St at SE corner,POINT (-73.8795549998336 40.84063900116436),2-5,"[-73.8795549998336, 40.84063900116436]",40.84063900116436,-73.8795549998336,10460
8,1742,http://web.mta.info/nyct/service/,Boston Rd & 178th St at NW corner,POINT (-73.87939700013239 40.84107800066419),2-5,"[-73.87939700013239, 40.84107800066419]",40.84107800066419,-73.87939700013239,10460
9,1743,http://web.mta.info/nyct/service/,Boston Rd & 174th St at SW corner,POINT (-73.88804799985908 40.83732500129732),2-5,"[-73.88804799985908, 40.83732500129732]",40.83732500129732,-73.88804799985908,10460


In [119]:
df_exerpt_subway['zipcode'] = df_exerpt_subway.latlong.apply(lambda x: zipcode_from_coord(x[1], x[0]))

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [85]:
df_subway['latlong'] = df_subway.latlong.str.split(' ')

In [86]:
df_subway['lat'] = df_subway['latlong'].apply(lambda x: x[1])

In [87]:
df_subway['long'] = df_subway['latlong'].apply(lambda x: x[0])

In [222]:
df_subway['zipcode'] = df_subway.latlong.apply(lambda x: coord_to_zip(x[1], x[0]))
#df_subway['zipcode'] = df_subway.iloc[1:10].latlong.apply(lambda x: coord_to_zip(x[1], x[0]))
#make sure it's not deleting everything besides the section currently being updated

In [230]:
df_subway.zipcode.nunique()

136

In [236]:
coord_to_zip(40.81958300055107,-73.90195799960925)

'10455'

In [247]:
df_test=df_subway.groupby('zipcode').count()

In [259]:
df_test.reset_index()

,zipcode,OBJECTID,URL,NAME,the_geom,LINE,latlong,lat,long
0,10001,51,51,50,51,51,51,51,51
1,10002,19,19,19,19,19,19,19,19
2,10003,28,28,28,28,28,28,28,28
3,10004,6,6,0,6,6,6,6,6
4,10006,30,30,25,30,30,30,30,30
5,10007,26,26,26,26,26,26,26,26
6,10009,4,4,4,4,4,4,4,4
7,10011,72,72,72,72,72,72,72,72
8,10012,34,34,31,34,34,34,34,34
9,10013,14,14,14,14,14,14,14,14


In [244]:
df_subway[df_subway.zipcode == '10001'].shape

(51, 9)

In [307]:
df_subway_count = df_test.reset_index()[['zipcode', 'OBJECTID']]

In [308]:
df_subway_count.rename(columns={'OBJECTID':'num_subway_entrances'},inplace=True)

In [309]:
df_subway_count.num_subway_entrances.unique()

array([51, 19, 28,  6, 30, 26,  4, 72, 34, 14,  5, 16, 27, 12,  3,  9, 10,
       17, 23, 24,  8, 18,  1,  7, 11, 33, 20, 21, 15, 44, 13, 41, 31, 29,
        2, 22])

In [310]:
bins = [-1,0, 6, 72]

In [311]:
bins_sub = pd.cut(df_subway_count.num_subway_entrances, bins)

In [312]:
df_subway_count['subway_bracket'] = bins_sub

In [315]:
df_subway_count.dtypes

zipcode                    int64
num_subway_entrances       int64
subway_bracket          category
dtype: object

In [314]:
df_subway_count.zipcode = df_subway_count.zipcode.astype(int)

In [291]:
df_subway_count[df_subway_count.num_subway_entrances == 1]

,zipcode,num_subway_entrances,subway_bracket
34,10111,1,"(0, 6]"
42,10171,1,"(0, 6]"
119,11379,1,"(0, 6]"
134,11693,1,"(0, 6]"


In [272]:
df_subway_count.num_subway_entrances.describe()

count    136.000000
mean      14.176471
std       10.651296
min        1.000000
25%        6.000000
50%       12.000000
75%       19.000000
max       72.000000
Name: num_subway_entrances, dtype: float64

In [342]:
df_master2 = pd.read_csv("df_master2.csv")

In [343]:
df_master2 = df_master2.merge(df_subway_count, how='left', on='zipcode')

In [327]:
df_master2.head()

,Unnamed: 0,zipcode,median_income,mean_income,population,City_zill,State_zill,Metro_zill,CountyName_zill,SizeRank_zill,...,2018-08_zill,2018-09_zill,2018-10_zill,2018-11_zill,2018-12_zill,2019-01_zill,restaurant_count,recycling_bin_count,num_subway_entrances,subway_bracket
0,0,10001,71244.6100,123112.7764,17678,New York,NY,New York-Newark-Jersey City,New York County,1297.0,...,3370.0,3348.0,3400.0,3370.0,3348.0,3400.0,488.0,1.0,51.0,"(6, 72]"
1,1,10002,30843.9574,46258.6118,70878,New York,NY,New York-Newark-Jersey City,New York County,9.0,...,3200.0,3199.0,3112.5,3112.5,3125.0,3050.0,509.0,10.0,19.0,"(6, 72]"
2,2,10003,89998.5339,139330.9968,53609,New York,NY,New York-Newark-Jersey City,New York County,45.0,...,3800.0,3800.0,3800.0,3800.0,3800.0,3750.0,682.0,7.0,28.0,"(6, 72]"
3,3,10004,110183.6857,156682.7615,1271,New York,NY,New York-Newark-Jersey City,New York County,3370.0,...,3972.5,3995.0,4110.0,4010.0,3995.0,3876.0,145.0,3.0,6.0,"(0, 6]"
4,4,10005,115133.2855,163762.6601,1517,New York,NY,New York-Newark-Jersey City,New York County,3250.0,...,3995.0,3995.0,3995.0,3910.0,3900.0,3910.0,72.0,3.0,NaN,NaN


In [344]:
df_master2.subway_bracket=df_master2.subway_bracket.cat.add_categories("0").fillna('0')

In [345]:
df_master2

,Unnamed: 0,zipcode,median_income,mean_income,population,City_zill,State_zill,Metro_zill,CountyName_zill,SizeRank_zill,...,2018-08_zill,2018-09_zill,2018-10_zill,2018-11_zill,2018-12_zill,2019-01_zill,restaurant_count,recycling_bin_count,num_subway_entrances,subway_bracket
0,0,10001,71244.6100,123112.7764,17678,New York,NY,New York-Newark-Jersey City,New York County,1297.0,...,3370.0,3348.0,3400.0,3370.0,3348.0,3400.0,488.0,1.0,51.0,"(6, 72]"
1,1,10002,30843.9574,46258.6118,70878,New York,NY,New York-Newark-Jersey City,New York County,9.0,...,3200.0,3199.0,3112.5,3112.5,3125.0,3050.0,509.0,10.0,19.0,"(6, 72]"
2,2,10003,89998.5339,139330.9968,53609,New York,NY,New York-Newark-Jersey City,New York County,45.0,...,3800.0,3800.0,3800.0,3800.0,3800.0,3750.0,682.0,7.0,28.0,"(6, 72]"
3,3,10004,110183.6857,156682.7615,1271,New York,NY,New York-Newark-Jersey City,New York County,3370.0,...,3972.5,3995.0,4110.0,4010.0,3995.0,3876.0,145.0,3.0,6.0,"(0, 6]"
4,4,10005,115133.2855,163762.6601,1517,New York,NY,New York-Newark-Jersey City,New York County,3250.0,...,3995.0,3995.0,3995.0,3910.0,3900.0,3910.0,72.0,3.0,NaN,0
5,5,10006,111220.0000,156776.0000,972,New York,NY,New York-Newark-Jersey City,New York County,3363.0,...,3500.0,3672.5,3550.0,3600.0,3700.0,3700.0,45.0,2.0,30.0,"(6, 72]"
6,6,10007,145458.7823,256235.5719,3520,New York,NY,New York-Newark-Jersey City,New York County,3354.0,...,5100.0,5342.0,5695.0,5795.0,5845.0,6325.0,141.0,9.0,26.0,"(6, 72]"
7,7,10009,56615.2777,78138.2679,56975,New York,NY,New York-Newark-Jersey City,New York County,22.0,...,3400.0,3495.0,3447.5,3320.0,3315.0,3350.0,311.0,13.0,4.0,"(0, 6]"
8,8,10010,93702.0061,137105.7060,27322,New York,NY,New York-Newark-Jersey City,New York County,749.0,...,3500.0,3500.0,3495.0,3497.5,3450.0,3500.0,259.0,0.0,NaN,0
9,9,10011,92359.4888,160936.5432,45899,New York,NY,New York-Newark-Jersey City,New York County,24.0,...,3750.0,3800.0,3800.0,3750.0,3688.5,3695.0,455.0,1.0,72.0,"(6, 72]"


In [346]:
df_master2.num_subway_entrances=df_master2.num_subway_entrances.fillna(0)

In [347]:
df_master2

,Unnamed: 0,zipcode,median_income,mean_income,population,City_zill,State_zill,Metro_zill,CountyName_zill,SizeRank_zill,...,2018-08_zill,2018-09_zill,2018-10_zill,2018-11_zill,2018-12_zill,2019-01_zill,restaurant_count,recycling_bin_count,num_subway_entrances,subway_bracket
0,0,10001,71244.6100,123112.7764,17678,New York,NY,New York-Newark-Jersey City,New York County,1297.0,...,3370.0,3348.0,3400.0,3370.0,3348.0,3400.0,488.0,1.0,51.0,"(6, 72]"
1,1,10002,30843.9574,46258.6118,70878,New York,NY,New York-Newark-Jersey City,New York County,9.0,...,3200.0,3199.0,3112.5,3112.5,3125.0,3050.0,509.0,10.0,19.0,"(6, 72]"
2,2,10003,89998.5339,139330.9968,53609,New York,NY,New York-Newark-Jersey City,New York County,45.0,...,3800.0,3800.0,3800.0,3800.0,3800.0,3750.0,682.0,7.0,28.0,"(6, 72]"
3,3,10004,110183.6857,156682.7615,1271,New York,NY,New York-Newark-Jersey City,New York County,3370.0,...,3972.5,3995.0,4110.0,4010.0,3995.0,3876.0,145.0,3.0,6.0,"(0, 6]"
4,4,10005,115133.2855,163762.6601,1517,New York,NY,New York-Newark-Jersey City,New York County,3250.0,...,3995.0,3995.0,3995.0,3910.0,3900.0,3910.0,72.0,3.0,0.0,0
5,5,10006,111220.0000,156776.0000,972,New York,NY,New York-Newark-Jersey City,New York County,3363.0,...,3500.0,3672.5,3550.0,3600.0,3700.0,3700.0,45.0,2.0,30.0,"(6, 72]"
6,6,10007,145458.7823,256235.5719,3520,New York,NY,New York-Newark-Jersey City,New York County,3354.0,...,5100.0,5342.0,5695.0,5795.0,5845.0,6325.0,141.0,9.0,26.0,"(6, 72]"
7,7,10009,56615.2777,78138.2679,56975,New York,NY,New York-Newark-Jersey City,New York County,22.0,...,3400.0,3495.0,3447.5,3320.0,3315.0,3350.0,311.0,13.0,4.0,"(0, 6]"
8,8,10010,93702.0061,137105.7060,27322,New York,NY,New York-Newark-Jersey City,New York County,749.0,...,3500.0,3500.0,3495.0,3497.5,3450.0,3500.0,259.0,0.0,0.0,0
9,9,10011,92359.4888,160936.5432,45899,New York,NY,New York-Newark-Jersey City,New York County,24.0,...,3750.0,3800.0,3800.0,3750.0,3688.5,3695.0,455.0,1.0,72.0,"(6, 72]"


In [348]:
df_master3 = df_master2[['zipcode', 'median_income', 
                         'population', 'CountyName_zill', '2019-01_zill', 'restaurant_count', 
                         'recycling_bin_count', 'num_subway_entrances', 'subway_bracket']]

In [354]:
df_master94=df_master3.dropna()

In [353]:
df_master3

,zipcode,median_income,population,CountyName_zill,2019-01_zill,restaurant_count,recycling_bin_count,num_subway_entrances,subway_bracket
0,10001,71244.6100,17678,New York County,3400.0,488.0,1.0,51.0,"(6, 72]"
1,10002,30843.9574,70878,New York County,3050.0,509.0,10.0,19.0,"(6, 72]"
2,10003,89998.5339,53609,New York County,3750.0,682.0,7.0,28.0,"(6, 72]"
3,10004,110183.6857,1271,New York County,3876.0,145.0,3.0,6.0,"(0, 6]"
4,10005,115133.2855,1517,New York County,3910.0,72.0,3.0,0.0,0
5,10006,111220.0000,972,New York County,3700.0,45.0,2.0,30.0,"(6, 72]"
6,10007,145458.7823,3520,New York County,6325.0,141.0,9.0,26.0,"(6, 72]"
7,10009,56615.2777,56975,New York County,3350.0,311.0,13.0,4.0,"(0, 6]"
8,10010,93702.0061,27322,New York County,3500.0,259.0,0.0,0.0,0
9,10011,92359.4888,45899,New York County,3695.0,455.0,1.0,72.0,"(6, 72]"


In [355]:
df_master94.to_csv('df_master94.csv')